In [97]:
import numpy as np
import matplotlib.pyplot as plt

# generate SEM
import sys
sys.path.append("..")

from notears.notears.notears import linear
from notears.notears.notears import utils
import helper.helper as h
import helper.methods as m

from importlib import reload  # Python 3.4+

### K-OMP Algorithm

In [152]:
def KOMP_Psi_K(Psi, K, max_coefs = 1e10, tol = 0.0, verbose = False, output = False, normalize = True):
    """Do Kernel OMP on Psi, K."""
    
    # get dimensions
    n = np.shape(Psi)[0]

    # initialize Lambda, idx, betas
    Lambda, idx, betas = [[] for _ in range(n)], [], np.zeros((n, n))

    # compute norms if we want to normalize
    norms = [1] * n # initialize as harmless 1 array
    if normalize: norms = [np.linalg.norm(x) for x in X.T]
    
    # for each possible coefficient
    for i in range(n ** 2):    
    
        # compute gains
        gains = np.abs([(k - betas.T @ Psi[i, :]) / norms[i] for i, k in enumerate(K)])
        
        # set forbidden set to -1, impossible to pick then
#         gains = gains.flatten()
#         gains[F] = - np.ones(len(F))
#         gains = gains.reshape(n, n)
    
        # print(gains)
        
        # stopping criterion
        if np.round(gains, 5).max() <= tol: 
            print("Break")
            break
    
        # append best atom to Lambda
        # if tie, pick the one that minimizes residual
        row, col = np.argmax(gains) // n, np.argmax(gains) % n
        Lambda[col].append(row)
    
        # check if we have a DAG, not super efficient
        if True: # h.is_dag(Lambda_to_adj(Lambda)): 
            # update only column col, use indices of 
            idx = Lambda[col]
            Psi_F = Psi[np.array(idx)[:, None], np.array(idx)[None, :]]
        
            # update betas
            betas[np.array(idx)[:, None], col] = np.linalg.inv(Psi_F) @ K[np.array(idx)[:, None], col]
    
        else:
            # append forbidden entry to forbidden list
            F.append(int(np.argmax(gains)))
            # remove coefficient from Lambda
            Lambda[col].remove(row)
    
        # print info if verbose
        if verbose:
            print(f"Iteration {i + 1}.\n")
            print(f"Gains:\n{np.round(gains, 3)}.\n")
            print(f"Beta_{i + 1}:\n{np.round(betas, 3)}.\n")
            print(f"Residual Squared: {np.round([Theta[i] - Psi[:, i] @ betas[:, i] for i in range(n)], 32)}.\n\n")

    return betas 

### Feature space $\Phi: \mathbb{R}^{T} \rightarrow \mathbb{R}^{T'}$

#### Polynomial Kernel with intercept.
$(1 + x^Tx)^k$.  

For $k = 0$, we have $T' = 1$.

For $k = 1$, we have $T' = T + 1$.

For $k = 2$, we have $T' = T + T (T + 1) / 2 + T + 1$.

#### Polynomial Kernel without intercept.
$(x^Tx)^k$.

For $k = 0$, we have $T' = 1$.

For $k = 1$, we have $T' = T$.

FOr $k = 2$, we have $T' = T + T (T + 1) / 2$. 

$$\sum_{i = 1}^k {T \choose i}$$.
#### Gaussian Kernel

#### Laplacian Kernel

In [99]:
from scipy.special import binom

In [100]:
T = 100
print(sum([binom(T, i) for i in range(1, 2)]))

100.0


#### Quadratic Polynomial

In [101]:
def poly_kernel_1(x):
    return x

def poly_kernel_2(x):
    x_2 = np.ones(T + int(T * (T - 1) / 2))
    
    x_2[:T] = x ** 2
    
    count = T
    for i in range(T):
        for j in range(i):
            x_2[count] = x[i] * x[j] * np.sqrt(2)
            count += 1
            
    return x_2

T = 100
X = np.array(range(T))

X_phi = poly_kernel_2()

TypeError: poly_kernel_2() missing 1 required positional argument: 'x'

In [102]:
def poly_kernel(x, k):
    
    L = sum([binom(T, i) for i in range(1, k)])
    x_phi = np.zeros(int(L))
    
    current = 0
    
    for i in range(1, k + 1):
        
        print("Filling polynomial degree", i)
        
        print(current, current + binom(T, i))
        # current = binom(T, i)
        
        # x_phi[current:current + binom(T, i)] = x ** k
        
        for j in range(int(binom(T, i))):
            print(j)
            x_phi[current + j] = 
        
        # x_phi[T:]

print(poly_kernel(1, 2))

SyntaxError: invalid syntax (<ipython-input-102-4e3a430b2f18>, line 19)

$$K(x, y) =\ <\phi(x), \phi(y)>\ =\ <(x, x^2), (y, y^2)>\ = x^Ty + (x^2)^Ty^2,$$
where $$\phi(x) = (x, x^2).$$

In [154]:
def lin_quad_phi(x):
    return np.append(x, x ** 2)

# x = np.random.rand(10, 1)
# y = np.random.rand(10)

T = 4
w = 3

x = np.array(range(T))

y_phi = w * lin_quad_phi(x)
y = w * x

print(y_phi[:T], (y_phi[:T] / np.sqrt(w)) ** 2)
print(lin_quad_phi(y))

Psi = np.dot(lin_quad_phi(x).T, lin_quad_phi(x))
K = np.dot(lin_quad_phi(x).T, y_phi)

K_2 = np.dot(lin_quad_phi(x).T, np.append(y_phi[:T], (y_phi[:T] / np.sqrt(w)) ** 2))

K_3 = np.dot(lin_quad_phi(x).T, lin_quad_phi(y))

print(K, K_2, K_3)
# print(1 / Psi * K)
# print(1 / Psi * K_2)
# print(1 / Psi * K_3)

[0 3 6 9] [ 0.  3. 12. 27.]
[ 0  3  6  9  0  9 36 81]
336 336.0 924
